In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)

2024-06-23 13:38:22.987521: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Versão do TensorFlow:  2.16.1


## Test GPU

In [5]:
tf.config.list_physical_devices()

2024-06-23 13:38:29.963539: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 13:38:30.022344: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 13:38:30.022383: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
# Action
# 0 Nothing
# 1 Go Left 
# 2 Go Right
# 3 RoteteLeft
# 4 RotateRight
# 5 TurnOnEngine

# Gerar os dados conforme especificado

data_nothing = []
data_rotate_left = []
data_rotate_right = []
data_go_right = []
data_go_left = []

# Preencher as listas de dados conforme as condições especificadas
for x in range(900):
    for currentX in range(900):
        for angle in range(-180, 181):
            if -90 <= angle <= -5:
                data_rotate_left.append([x, currentX, angle, 3])  # RotateLeft
            elif 5 <= angle <= 90:
                data_rotate_right.append([x, currentX, angle, 4])  # RotateRight
            elif -5 < angle < 5:
                data_nothing.append([x, currentX, angle, 0])  # Nothing
            else:
                if currentX < x:
                    data_go_right.append([x, currentX, angle, 2])  # Go Right
                else:
                    data_go_left.append([x, currentX, angle, 1])  # Go Left

# Determinar o tamanho mínimo entre as listas para balanceamento
min_size = min(len(data_nothing), len(data_rotate_left), len(data_rotate_right), len(data_go_right), len(data_go_left))

# Balancear as listas
balanced_data = (
    data_nothing[:min_size] +
    data_rotate_left[:min_size] +
    data_rotate_right[:min_size] +
    data_go_right[:min_size] +
    data_go_left[:min_size]
)

# Contar a quantidade de cada tipo
count_nothing = sum(1 for item in balanced_data if item[3] == 0)
count_rotate_left = sum(1 for item in balanced_data if item[3] == 3)
count_rotate_right = sum(1 for item in balanced_data if item[3] == 4)
count_go_right = sum(1 for item in balanced_data if item[3] == 2)
count_go_left = sum(1 for item in balanced_data if item[3] == 1)

# Mostrar a quantidade de cada tipo
print("Quantidade de 'Nothing':", count_nothing)
print("Quantidade de 'RotateLeft':", count_rotate_left)
print("Quantidade de 'RotateRight':", count_rotate_right)
print("Quantidade de 'Go Right':", count_go_right)
print("Quantidade de 'Go Left':", count_go_left)

balanced_data = np.array(balanced_data)
X = balanced_data[:, :3]  # Features: x, currentX e angle
y = balanced_data[:, 3]   # Labels: 0, 1, 2, 3 ou 4

len(balanced_data)

Quantidade de 'Nothing': 7290000
Quantidade de 'RotateLeft': 7290000
Quantidade de 'RotateRight': 7290000
Quantidade de 'Go Right': 7290000
Quantidade de 'Go Left': 7290000


36450000

In [17]:

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [32]:

# Salvar os dados em um arquivo .pkl
with open('data.pkl', 'wb') as f:
    pickle.dump([X_train, y_train, X_test, y_test], f)
!ls

data.pkl  data2.pkl  logs  models  notebooks  tensorflow-tutorials


In [27]:
# Carregar os dados do arquivo .pkl
with open('./data.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './data2.pkl'

In [8]:
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # 5 saídas: 0, 1, 2, 3, 4
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=100, batch_size=32000, validation_split=0.2, callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-23 13:38:55.356129: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 13:38:55.356254: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 13:38:55.356293: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have bee

Epoch 1/100


I0000 00:00:1719149939.384047     607 service.cc:145] XLA service 0x7f027c004b70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719149939.384114     607 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-06-23 13:38:59.427494: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-23 13:38:59.617041: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1719149940.928108     656 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_197', 32 bytes spill stores, 40 bytes spill loads

I0000 00:00:1719149945.019972     660 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_547', 12 bytes spill stores, 12 bytes spill loads



 21/729 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.3274 - loss: 10.3755    

I0000 00:00:1719149947.285966     607 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


724/729 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8305 - loss: 1.0872

I0000 00:00:1719149953.912241     839 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_39', 60 bytes spill stores, 60 bytes spill loads



729/729 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.8313 - loss: 1.0809 - val_accuracy: 0.9792 - val_loss: 0.0613
Epoch 2/100
729/729 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9721 - loss: 0.0801 - val_accuracy: 0.9899 - val_loss: 0.0370
Epoch 3/100
729/729 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9858 - loss: 0.0420 - val_accuracy: 0.9658 - val_loss: 0.0868
Epoch 4/100
729/729 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9823 - loss: 0.0496 - val_accuracy: 0.9924 - val_loss: 0.0274
Epoch 5/100
729/729 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9923 - loss: 0.0263 - val_accuracy: 0.9925 - val_loss: 0.0217
Epoch 6/100
729/729 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9915 - loss: 0.0236 - val_accuracy: 0.9963 - val_loss: 0.0165
Epoch 7/100
729/729 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9934 - loss: 0.0195 - val_accuracy: 0.9962 - val_loss: 0.0140
Epoch 8/100
729/729 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9935 - loss: 0.0185 - val_accuracy: 0.99

In [9]:

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')


227813/227813 ━━━━━━━━━━━━━━━━━━━━ 408s 2ms/step - accuracy: 0.9983 - loss: 0.0052
Model Accuracy: 99.83%
227813/227813 ━━━━━━━━━━━━━━━━━━━━ 316s 1ms/step


In [16]:
 


targetX = 0
agentX = 0
angle = 45

predictions = model.predict(np.array([[targetX, agentX, angle]]))
# 0 Nothing
# 1 Go Left
# 2 Go Right
# 3 RoteteLeft
# 4 RotateRight
# 5 TurnOnEngine
class_names = ['Nothing', 'Go Left', 'Go Right',
               'RoteteLeft', 'RotateRight', 'TurnOnEngine']

score = tf.nn.softmax(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
This image most likely belongs to RotateRight with a 40.46 percent confidence.


In [47]:
%tensorboard

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:

Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

In [48]:
x__training,x__test,y__training,y__test=train_test_split(x_,y_,test_size=.percent,random_state=0)
with open('.pkl',mode='wb') as f:
     pickle.dump([x__training,y__training,x__test,y__test],f)

SyntaxError: invalid syntax (1265863735.py, line 1)

In [64]:
model.save("./models/model_saved.h5")
model.save(filepath="./models/model_saved.keras")


 

#### The command below is to copy from the docker container to local folder

In [44]:
!docker cp cool_spence:/tf/models/. /mnt/d/git/victorbarbosa/src/assets/models/

/usr/bin/sh: 1: docker: not found


### From here to you only MACOS to test 

In [20]:
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model("./models/model_saved.keras")

# targetX = 0
# agentX = 0
# angle = -2

# predictions = model.predict([[targetX, agentX, angle]])
# # 0 Nothing
# # 1 Go Left
# # 2 Go Right
# # 3 RoteteLeft
# # 4 RotateRight
# # 5 TurnOnEngine
# class_names = ['Nothing', 'Go Left', 'Go Right',
#                'RoteteLeft', 'RotateRight', 'TurnOnEngine']

# score = tf.nn.softmax(predictions[0])
# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(class_names[np.argmax(score)], 100 * np.max(score))
# )

In [14]:
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('./models/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, './models/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model './models/saved_model/' web_model

INFO:tensorflow:Assets written to: ./models/saved_model/assets


INFO:tensorflow:Assets written to: ./models/saved_model/assets
